In [ ]:
import os
import json
import glob

# input and output directory
transformed_dir = "../../BoT-SORT_outputs/Outdoor/transformed"
output_dir = os.path.join(transformed_dir, "with_jersey_number")
os.makedirs(output_dir, exist_ok=True)

# illegible.json の読み込み
with open("../../jersey-number-pipeline/out/OutdoorResults/illegible.json", "r", encoding="utf-8") as f:
    illegible_data = json.load(f).get("illegible", {})
print(f"illegible.json: {len(illegible_data)} images")

# final_results.json の読み込み
with open("../../jersey-number-pipeline/out/OutdoorResults/final_results.json", "r", encoding="utf-8") as f:
    final_results = json.load(f)
print(f"final_results.json: {len(final_results)} images")

# transformed フォルダ内の *.txt ファイルを取得（with_jersey_numberフォルダは除く）
txt_files = glob.glob(os.path.join(transformed_dir, "*.txt"))

for txt_file in txt_files:
    base_name = os.path.splitext(os.path.basename(txt_file))[0]
    
    # 対象画像の final_results 情報（存在しない場合は空の dict）
    final_result_for_image = final_results.get(base_name, {})
    
    # 対象画像の illegible 情報（除外すべき track リスト、存在しなければ空リスト）
    illegible_tracks = illegible_data.get(base_name, [])
    
    output_lines = []
    with open(txt_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # 行をカンマ区切りで分割（フレーム番号, track_id, x, y）
            parts = line.split(",")
            if len(parts) < 4:
                continue
            frame, track_id, x, y = parts[0], parts[1], parts[2], parts[3]
            track_key = "track" + track_id  # final_results のキーは "trackX" の形式
            
            # illegible に含まれている track は除外
            if track_key in illegible_tracks:
                continue
            
            # final_results に該当 track が存在し、かつジャージ番号が -1 でなければ取得
            jersey_number = final_result_for_image.get(track_key)
            if jersey_number is None or jersey_number == -1 or jersey_number == "-1":
                continue
            
            # 新たな列としてジャージ番号を追加
            new_line = ",".join([frame, track_id, x, y, str(jersey_number)])
            output_lines.append(new_line)
            
    # 出力先ファイルに保存
    output_file = os.path.join(output_dir, os.path.basename(txt_file))
    with open(output_file, "w", encoding="utf-8") as f:
        for line in output_lines:
            f.write(line + "\n")
    print(f"Saved: {output_file}")


illegible.json: 96 images
final_results.json: 97 images
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0104_1.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0104_2.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0104_3.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0104_4.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0104_5.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0104_6.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0104_7.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0105_10.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0105_11.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0105_12.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/IMG_0105_1.txt
Saved: ../BoT-SORT_outputs/Outdoor/transformed/wit

In [ ]:
import math
import numpy as np
from scipy.spatial.distance import jensenshannon

def compute_js(p, q, eps=1e-10):
    """
    Calculate the JS divergence (JS distance after normalization) using SciPy's jensenshannon.
    The input histograms are normalized internally.
    """
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)
    p = p / (np.sum(p) + eps)
    q = q / (np.sum(q) + eps)
    return jensenshannon(p, q)

# ------------------------------
# 1. Input/output directory settings
# ------------------------------
# Directory of numbered txt files (pre-processed)
jersey_dir = "../../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number"
# Output destination given team (offense/defense) and jersey number
output_dir = os.path.join(jersey_dir, "with_team")
os.makedirs(output_dir, exist_ok=True)

# Directory for ground_truth determination
free_throw_gt_dir = "../../ground_truth/Outdoor/MOT_files/split_transformed/free_throw"
check_ball_gt_dir = "../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball"

jersey_files = glob.glob(os.path.join(jersey_dir, "*.txt"))
for file_path in jersey_files:
    basename = os.path.basename(file_path)
    print("----- Processing file:", basename, "-----")
    if os.path.exists(os.path.join(free_throw_gt_dir, basename)):
        designated_point = (752.5, 580)
        is_top = False
        print("-> Detected free_throw file. Designated point:", designated_point)
    elif os.path.exists(os.path.join(check_ball_gt_dir, basename)):
        designated_point = (752.5, 1105)
        is_top = True
        print("-> Detected top file. Designated point:", designated_point)
    else:
        print("-> File not found in ground_truth directories. Skipping.")
        continue

    if basename.startswith("IMG_") and basename.endswith(".txt"):
        identifier = basename[len("IMG_"):-len(".txt")]
    else:
        identifier = os.path.splitext(basename)[0]

    # ------------------------------
    # 2. File loading & grouping by frame
    # ------------------------------
    # Each line is in the format “frame,track_id,x,y,jersey_number
    records_by_frame = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(",")
            if len(parts) < 5:
                continue
            frame = int(parts[0])
            track = parts[1]
            x = float(parts[2])
            y = float(parts[3])
            jersey = parts[4]
            rec = {"frame": frame+1, "track": track, "x": x, "y": y, "jersey": jersey}
            records_by_frame.setdefault(frame, []).append(rec)
    frames = sorted(records_by_frame.keys())
    if len(frames) == 0:
        print("-> No frames found. Skipping file.")
        continue

    # ------------------------------
    # 2-1. Team assignment in the first frame
    # ------------------------------
    track_assignment = {}
    first_frame = frames[0]
    first_records = records_by_frame[first_frame]
    print("-> First frame (frame {}) has {} tracks.".format(first_frame, len(first_records)))
    if len(first_records) < 6:
        print("WARNING: First frame has less than 6 tracks. Expected 6 for proper team assignment.")
    elif len(first_records) > 6:
        first_records = sorted(first_records, key=lambda r: math.sqrt((r["x"] - designated_point[0])**2 + (r["y"] - designated_point[1])**2))[:6]
        print("-> More than 6 tracks found. Using top 6 by designated-point distance.")

    if not is_top:
        # free_throw の場合
        for rec in first_records:
            rec["distance"] = math.sqrt((rec["x"] - designated_point[0])**2 + (rec["y"] - designated_point[1])**2)
        sorted_by_dist = sorted(first_records, key=lambda r: r["distance"])
        initial_offense = sorted_by_dist[0]
        initial_offense_id = initial_offense["track"]
        track_assignment[initial_offense_id] = "Offense"
        print("free_throw: Initial offense candidate:", initial_offense)
        hist_dir = os.path.join("../../color_histograms/Outdoor", identifier)
        offense_hist_path = os.path.join(hist_dir, f"{identifier}_track{initial_offense_id}.npy")
        if os.path.exists(offense_hist_path):
            offense_hist = np.load(offense_hist_path)
        else:
            offense_hist = None
            print("WARNING: Offense histogram not found for track", initial_offense_id)
        divergence_list = []
        if offense_hist is not None:
            for rec in first_records:
                tid = rec["track"]
                if tid == initial_offense_id:
                    continue
                hist_path = os.path.join(hist_dir, f"{identifier}_track{tid}.npy")
                if not os.path.exists(hist_path):
                    print("WARNING: Histogram not found for track", tid)
                    continue
                hist = np.load(hist_path)
                d = compute_js(offense_hist, hist)
                divergence_list.append((tid, d))
                print("free_throw: JS divergence between track", initial_offense_id, "and track", tid, "=", d)
            divergence_list.sort(key=lambda x: x[1])
            for tid, d in divergence_list[:2]:
                track_assignment[tid] = "Offense"
                print("free_throw: Additional offense candidate selected (by JS): track", tid, "divergence =", d)
        else:
            print("free_throw: Offense histogram unavailable, skipping JS selection.")
        for rec in first_records:
            tid = rec["track"]
            if tid not in track_assignment:
                track_assignment[tid] = "Defense"
        print("free_throw: First frame assignment:", track_assignment)
    else:
        # top の場合
        for rec in first_records:
            rec["distance"] = math.sqrt((rec["x"] - designated_point[0])**2 + (rec["y"] - designated_point[1])**2)
        sorted_by_dist = sorted(first_records, key=lambda r: r["distance"])
        initial_offense = sorted_by_dist[0]
        initial_offense_id = initial_offense["track"]
        track_assignment[initial_offense_id] = "Offense"
        print("top: Initial offense candidate:", initial_offense)
        remaining = [r for r in first_records if r["track"] != initial_offense_id]
        for rec in remaining:
            rec["dist_to_offense"] = math.sqrt((rec["x"] - initial_offense["x"])**2 + (rec["y"] - initial_offense["y"])**2)
        sorted_by_offense_dist = sorted(remaining, key=lambda r: r["dist_to_offense"])
        initial_defense = sorted_by_offense_dist[0]
        initial_defense_id = initial_defense["track"]
        track_assignment[initial_defense_id] = "Defense"
        print("top: Initial defense candidate (closest to offense):", initial_defense)
        hist_dir = os.path.join("../../color_histograms/Outdoor", identifier)
        offense_hist_path = os.path.join(hist_dir, f"{identifier}_track{initial_offense_id}.npy")
        if os.path.exists(offense_hist_path):
            offense_hist = np.load(offense_hist_path)
        else:
            offense_hist = None
            print("WARNING: Offense histogram not found for track", initial_offense_id)
        defense_hist_path = os.path.join(hist_dir, f"{identifier}_track{initial_defense_id}.npy")
        if os.path.exists(defense_hist_path):
            defense_hist = np.load(defense_hist_path)
        else:
            defense_hist = None
            print("WARNING: Defense histogram not found for track", initial_defense_id)
        remaining_ids = set(r["track"] for r in first_records if r["track"] not in track_assignment)
        offense_divs = []
        if offense_hist is not None:
            for tid in remaining_ids:
                hist_path = os.path.join(hist_dir, f"{identifier}_track{tid}.npy")
                if not os.path.exists(hist_path):
                    print("WARNING: Histogram not found for track", tid)
                    continue
                hist = np.load(hist_path)
                d = compute_js(offense_hist, hist)
                offense_divs.append((tid, d))
                print("top: JS divergence between offense track", initial_offense_id, "and track", tid, "=", d)
            offense_divs.sort(key=lambda x: x[1])
            for tid, d in offense_divs[:2]:
                track_assignment[tid] = "Offense"
                print("top: Additional offense candidate selected (by JS): track", tid, "divergence =", d)
        else:
            print("top: Offense histogram unavailable, skipping offense JS selection.")
        defense_divs = []
        if defense_hist is not None:
            for tid in remaining_ids:
                if tid in track_assignment:
                    continue
                hist_path = os.path.join(hist_dir, f"{identifier}_track{tid}.npy")
                if not os.path.exists(hist_path):
                    print("WARNING: Histogram not found for track", tid)
                    continue
                hist = np.load(hist_path)
                d = compute_js(defense_hist, hist)
                defense_divs.append((tid, d))
                print("top: JS divergence between defense track", initial_defense_id, "and track", tid, "=", d)
            defense_divs.sort(key=lambda x: x[1])
            for tid, d in defense_divs[:2]:
                track_assignment[tid] = "Defense"
                print("top: Additional defense candidate selected (by JS): track", tid, "divergence =", d)
        else:
            print("top: Defense histogram unavailable, skipping defense JS selection.")
        for rec in first_records:
            tid = rec["track"]
            if tid not in track_assignment:
                track_assignment[tid] = "Defense"
        print("top: First frame assignment:", track_assignment)

    # ------------------------------
    # 2-3. 2フレーム目以降：新規出現トラックの割り当て
    # ------------------------------
    for frame in frames[1:]:
        current_records = records_by_frame[frame]
        offense_count = sum(1 for rec in current_records if rec["track"] in track_assignment and track_assignment[rec["track"]] == "Offense")
        defense_count = sum(1 for rec in current_records if rec["track"] in track_assignment and track_assignment[rec["track"]] == "Defense")
        for rec in current_records:
            tid = rec["track"]
            if tid in track_assignment:
                continue
            if offense_count <= 2 or defense_count <= 2:
                if offense_count <= 2 and defense_count <= 2:
                    assignment = "Offense" if offense_count <= defense_count else "Defense"
                elif offense_count <= 2:
                    assignment = "Offense"
                else:
                    assignment = "Defense"
            else:
                assignment = "Offense" if offense_count <= defense_count else "Defense"
            track_assignment[tid] = assignment
            if assignment == "Offense":
                offense_count += 1
            else:
                defense_count += 1
            print("Frame", frame, ": New track", tid, "assigned to", assignment)
    print("-> Final assignment across frames:", track_assignment)

    # ------------------------------
    # 3. 各トラックの出現期間・フレーム数を記録
    # ------------------------------
    track_info = {}
    for frame in frames:
        for rec in records_by_frame[frame]:
            tid = rec["track"]
            role = track_assignment.get(tid, "")
            jersey = rec["jersey"]
            if tid not in track_info:
                track_info[tid] = {"start": frame, "end": frame, "count": 1, "jersey": jersey, "role": role}
            else:
                track_info[tid]["end"] = frame
                track_info[tid]["count"] += 1

    # ------------------------------
    # 3.5 各フレームで各チームのトラック数が3つ以下になるように調整
    # 出力前に、各フレーム内の記録をチェックし、同一チームのトラックが3を超えている場合、
    # track_info の出現フレーム数が少ない（短い）トラックを除外して、3件に調整する。
    # ------------------------------
    for frame in frames:
        frame_records = records_by_frame[frame]
        team_to_tracks = {}
        for rec in frame_records:
            tid = rec["track"]
            role = track_assignment.get(tid, "")
            team_to_tracks.setdefault(role, set()).add(tid)
        for role, tids in team_to_tracks.items():
            if role not in ["Offense", "Defense"]:
                continue
            if len(tids) > 3:
                sorted_tids = sorted(list(tids), key=lambda t: track_info[t]["count"], reverse=True)
                #sorted_tids = sorted(list(tids), key=lambda t: track_info[t]["start"], reverse=True)
                keep = set(sorted_tids[:3])
                new_frame_records = [rec for rec in frame_records if (track_assignment.get(rec["track"], "") != role) or (rec["track"] in keep)]
                records_by_frame[frame] = new_frame_records
                print(f"Frame {frame}: For role {role}, reduced tracks from {len(tids)} to 3 based on track length.")

    # ------------------------------
    # 4. 出力用に各レコードの背番号を更新（チーム情報の接頭辞付与）
    # ------------------------------
    for frame in frames:
        for rec in records_by_frame[frame]:
            tid = rec["track"]
            role = track_assignment.get(tid, "")
            jersey = track_info.get(tid, {}).get("jersey", rec["jersey"])
            if role == "Offense":
                rec["jersey"] = "O" + str(jersey)
            elif role == "Defense":
                rec["jersey"] = "D" + str(jersey)
            else:
                rec["jersey"] = str(jersey)
    
    # ------------------------------
    # 5. ファイル出力：各行は "frame,track,x,y,統合背番号" の形式
    # ------------------------------
    output_lines = []
    for frame in frames:
        for rec in records_by_frame[frame]:
            line = f'{rec["frame"]},{rec["track"]},{rec["x"]},{rec["y"]},{rec["jersey"]}'
            output_lines.append(line)
    
    out_path = os.path.join(output_dir, basename)
    with open(out_path, "w", encoding="utf-8") as f:
        for line in output_lines:
            f.write(line + "\n")
    print("-> Output file written to:", out_path, "\n")